ข้อมูลที่ publish เข้าสู่ Kafka broker คือ 2022-01-01-2022-12-31.csv ซึ่งมีหน้าตาแบบนี้

{"locationId": "225577", "location": "Bank of Ayuthaya Head Office Yan Nawa, Bangkok", "parameter": "pm25", "value": "35.0", "date": "{'utc': '2022-12-31T04:00:00+00:00', 'local': '2022-12-31T11:00:00+07:00'}", "unit": "\u00b5g/m\u00b3", "coordinates": "{'latitude': 13.679226, 'longitude': 100.54687}", "country": "NA", "city": "", "isMobile": "False", "isAnalysis": "", "entity": "Governmental Organization", "sensorType": "reference grade"}
{"locationId": "225577", "location": "Bank of Ayuthaya Head Office Yan Nawa, Bangkok", "parameter": "pm25", "value": "36.0", "date": "{'utc': '2022-12-31T05:00:00+00:00', 'local': '2022-12-31T12:00:00+07:00'}", "unit": "\u00b5g/m\u00b3", "coordinates": "{'latitude': 13.679226, 'longitude': 100.54687}", "country": "NA", "city": "", "isMobile": "False", "isAnalysis": "", "entity": "Governmental Organization", "sensorType": "reference grade"}
{"locationId": "225577", "location": "Bank of Ayuthaya Head Office Yan Nawa, Bangkok", "parameter": "pm25", "value": "36.0", "date": "{'utc': '2022-12-31T06:00:00+00:00', 'local': '2022-12-31T13:00:00+07:00'}", "unit": "\u00b5g/m\u00b3", "coordinates": "{'latitude': 13.679226, 'longitude': 100.54687}", "country": "NA", "city": "", "isMobile": "False", "isAnalysis": "", "entity": "Governmental Organization", "sensorType": "reference grade"}

In [1]:
# จำเป็นต้องเปลี่ยนเลขไอพีของ Kafka broker โดยใช้ INTERNAL IP Address พร้อมด้วยหมายเลข port 9092

kafka_broker = "10.128.0.12:9092"

In [2]:
# สร้าง Spark session โดยใช้การตั้งค่าที่กำหนด

from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("KafkaSubscribe").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1000m").\
        config("spark.executor.cores", "2").\
        config("spark.cores.max", "6").\
        config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0").\
        getOrCreate()



Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-145da86b-af2d-4abc-8707-707ba1bc1195;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 654ms :: artifacts dl 13

# 1. ได้รับ Unbounded input table 

In [3]:
# สร้าง DataFrame โดยอ่านข้อมูลจาก Kafka stream

unboundInput_table_df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", kafka_broker) \
  .option("subscribe", "quickstart-events") \
  .option("group.id", "Aekanun-Spark-App") \
  .load()

In [4]:
# แสดง schema ของ column ต่างๆ ใน DataFrame

unboundInput_table_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



# 2. เกิดการ Query - Stream Data Processing

# 3. ได้รับ Result table

In [5]:
result_table_df = unboundInput_table_df

In [6]:
result_table_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



# 4. ได้รับ Output (table)

In [ ]:
from pyspark.sql import functions as F



output_df = result_table_df.writeStream \
    .format("console") \
    .outputMode("append") \
    .trigger(processingTime='5 seconds') \
    .start()

output_df.awaitTermination()


23/06/22 00:55:48 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-1aeaf0ac-b216-450d-ad5c-1bbd0642d5dc. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+-----+-----+---------+------+---------+-------------+
|key|value|topic|partition|offset|timestamp|timestampType|
+---+-----+-----+---------+------+---------+-------------+
+---+-----+-----+---------+------+---------+-------------+



23/06/22 00:55:53 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 5139 milliseconds


-------------------------------------------
Batch: 1
-------------------------------------------
+----+--------------------+-----------------+---------+------+--------------------+-------------+
| key|               value|            topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-----------------+---------+------+--------------------+-------------+
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0| 23607|2023-06-22 00:56:...|            0|
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0| 23608|2023-06-22 00:56:...|            0|
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0| 23609|2023-06-22 00:56:...|            0|
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0| 23610|2023-06-22 00:56:...|            0|
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0| 23611|2023-06-22 00:56:...|            0|
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0| 23612|2023-06-22 00:56:...|            0|
|null|[7B 22 6C 6F 63

-------------------------------------------
Batch: 2
-------------------------------------------


+----+--------------------+-----------------+---------+------+--------------------+-------------+
| key|               value|            topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-----------------+---------+------+--------------------+-------------+
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0| 26101|2023-06-22 00:56:...|            0|
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0| 26102|2023-06-22 00:56:...|            0|
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0| 26103|2023-06-22 00:56:...|            0|
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0| 26104|2023-06-22 00:56:...|            0|
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0| 26105|2023-06-22 00:56:...|            0|
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0| 26106|2023-06-22 00:56:...|            0|
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0| 26107|2023-06-22 00:56:...|            0|
|null|[7B 22 6C 6F 6

In [ ]:
output_df.stop()